In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
import datetime 

import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import sqlalchemy

init_notebook_mode(connected=True)

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

WINDOW = 22

Using TensorFlow backend.


In [2]:
"""
Парсинг данных

"""
def connect(user, password, db, host: str, port: int, echo=True):
    url = 'postgresql+psycopg2://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)
    eng = sqlalchemy.create_engine(url, client_encoding='utf8', echo=echo)
    meta = sqlalchemy.MetaData(bind=eng)

    return eng, meta


def get_data_frame(pair: str = 'USDT_BTC', exchange: str = 'poloniex') -> pd.DataFrame:
    """Метод стягивания данных из базы в датафрейм.
    По умолчанию тянет все значения в базе для валютной пары доллар биткоин.
    Список спаршенных пар смотри в таблице Pair
    Цепляться будет отовсюду где есть инетрнет"""
    engine, meta = connect(user='postgres', password='password', db='btccandles', host='94.230.125.199', port=16432)
    df = pd.read_sql_query(
        "SELECT date, time, open, close, low, high, volume, pair.\"name\""
        "FROM candlestick, pair WHERE candlestick.pair_id = pair.id AND pair.id IN ("
        "SELECT pair.id FROM pair, exchange WHERE ("
        "SELECT pair.id FROM pair WHERE pair.name = '" + pair +
        "') = pair.alias_id AND pair.exchange_id = (SELECT exchange.id FROM exchange WHERE exchange.\"name\" = '" + exchange + "'));",
        con=engine)
    return df

"""
Загрузка данных
"""
def load_data(X, seq_len, train_size=1):
    # Определяем число фич
    amount_of_features = X.shape[1] 
    
    sequence_length = seq_len + 1 
    data = []
    
    # Бьем тренировочные данные на блоки по размеру окна
    for index in range(len(X) - sequence_length):
        data.append(X[index: index + sequence_length])
    
    data = np.array(data)
    train_split = int(round(train_size * data.shape[0]))
    train_data = data[:train_split, :]
    
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1][:,-1]
    
    # Меняем размер входного фрейма на [dim, window, featureNumber]
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    return x_train, y_train

"""
Собираем LSTM
"""

def build_model(input_shape):
    d = 0.2
    model = Sequential()
    
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(d))
        
    model.add(LSTM(128, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(d))
        
    model.add(Dense(32,kernel_initializer="normal",activation='relu'))        
    model.add(Dense(1,kernel_initializer="normal",activation='linear'))
    
    model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
    
    return model

def predictNumDay(num, pathModel, data, scaler):
    m = load_model(pathModel)
    
    prediction = []
    lastbatch = (data[-WINDOW:])
    for i in np.arange(num):    
        res = m.predict([lastbatch.reshape(1,22, 1)])
        prediction.append(scaler.inverse_transform(res))
        lastbatch = np.concatenate([lastbatch[1:],res])
        m.train_on_batch(lastbatch.reshape(1,22,1), res)
           
    return np.array(prediction).reshape(num)   

In [3]:
"""
typeBlockchain:
USDT_BTC
USDT_LTC
USDT_ETH
USDT_ETC
USDT_XRP
"""

def nextDayPrediction(typeBlockchain, N = 1):    
    
    df = get_data_frame(typeBlockchain)

    scaler = MinMaxScaler()


    x = df['close'].copy()
    y = df['close'].copy()

    x = scaler.fit_transform(x.values.reshape(-1, 1))

    y = scaler.fit_transform(y.values.reshape(-1, 1))
    
    X_train, y_train = load_data(x, WINDOW)
    
    #print (X_train.shape, y_train.shape)
    
    model = build_model(input_shape=(WINDOW, 1))
    
    print('START FIT MODEL...')
    
    start = time.time()
    model.fit(X_train, y_train, batch_size=32, epochs=500,verbose=0)
    end = time.time()

    print ('Learning time: ', end-start)
    
    today = time.strftime("_%d_%m_%Y")
    
    pathModel = "../models/model_1f_" + typeBlockchain + today +".h5"
    save_model(model, pathModel)
    
    #model = load_model(pathModel)
    
    trainPredict = model.predict(X_train)
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([y_train])

    trainScore = metrics.mean_squared_error(trainY[0], trainPredict[:,0]) ** .5
    print('Train Score: %.2f RMSE' % (trainScore))
    
    prices = df.close.values.astype('float32')
    prices = prices.reshape(len(prices), 1)
    
    trainPredictPlot = np.empty_like(prices)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[WINDOW:len(trainPredict)+WINDOW, :] = trainPredict

    """
    
    plt.plot(pd.DataFrame(prices, columns=["close"], index=df.index).close, label='Actual')
    plt.plot(pd.DataFrame(trainPredictPlot, columns=["close"], index=df.index).close, label='Training')
    plt.legend(loc='best')
    plt.show()
    
    """
    
    lastDate =str(df.date[df.last_valid_index()]).split('-')
    currentData = datetime.date(int(lastDate[0]),int(lastDate[1]),int(lastDate[2])+1)
    predictionDate = pd.date_range(currentData,periods=N)
    predictNday =  (predictNumDay(N, pathModel, x, scaler))

    prediction = pd.DataFrame(predictNday, columns=["predictionPrice"], index = predictionDate.values)
    
    Actual = pd.DataFrame(prices, columns=["close"], index=df.date).close
    Training = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close
    pred = pd.DataFrame(trainPredictPlot, columns=["close"], index=df.date).close

    """ActualValues = go.Scatter( x = df.date, y = Actual, name = 'ActualValues')
    TrainingValues = go.Scatter( x = df.date, y = Training, name = 'TrainingValues')

    iplot([ActualValues,TrainingValues])
    
    our_Predict = go.Scatter( x = prediction.index, y = prediction.predictionPrice, name = 'Next5DayValues')

    iplot([our_Predict, ActualValues])"""
    
    return prediction

In [4]:
USDT_BTC = nextDayPrediction('USDT_BTC', N = 1)

2017-07-20 21:46:42,975 INFO sqlalchemy.engine.base.Engine select version()
2017-07-20 21:46:42,976 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 21:46:42,978 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-20 21:46:42,979 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 21:46:42,981 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-20 21:46:42,982 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 21:46:42,983 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-20 21:46:42,984 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 21:46:42,985 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-20 21:46:42,987 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 21:46:42,988 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [5]:
USDT_LTC = nextDayPrediction('USDT_LTC', N = 1)

2017-07-20 22:03:44,632 INFO sqlalchemy.engine.base.Engine select version()
2017-07-20 22:03:44,633 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:03:44,634 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-20 22:03:44,635 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:03:44,636 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:03:44,637 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:03:44,638 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:03:44,639 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:03:44,640 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-20 22:03:44,640 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:03:44,642 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [6]:
USDT_ETH = nextDayPrediction('USDT_ETH', N = 1)

2017-07-20 22:14:09,693 INFO sqlalchemy.engine.base.Engine select version()
2017-07-20 22:14:09,694 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:14:09,698 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-20 22:14:09,698 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:14:09,701 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:14:09,702 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:14:09,703 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:14:09,704 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:14:09,705 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-20 22:14:09,705 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:14:09,707 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [7]:
USDT_ETC = nextDayPrediction('USDT_ETC', N = 1)

2017-07-20 22:35:07,242 INFO sqlalchemy.engine.base.Engine select version()
2017-07-20 22:35:07,242 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:35:07,244 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-20 22:35:07,244 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:35:07,246 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:35:07,246 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:35:07,248 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:35:07,248 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:35:07,250 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-20 22:35:07,250 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:35:07,251 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

In [9]:
USDT_XRP = nextDayPrediction('USDT_XRP', N = 1)

2017-07-20 22:59:06,587 INFO sqlalchemy.engine.base.Engine select version()
2017-07-20 22:59:06,588 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:59:06,590 INFO sqlalchemy.engine.base.Engine select current_schema()
2017-07-20 22:59:06,590 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:59:06,592 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:59:06,593 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:59:06,595 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-07-20 22:59:06,595 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:59:06,597 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2017-07-20 22:59:06,598 INFO sqlalchemy.engine.base.Engine {}
2017-07-20 22:59:06,599 INFO sqlalchemy.engine.base.Engine select date, time, open, close, low, high, volume, pair."name" from candlestick, pair where candlestick.pair_id=pair.id and pair."name"='USDT_

ResourceExhaustedError: OOM when allocating tensor with shape[128,128]
	 [[Node: gradients_9/lstm_12/while/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, _class=["loc:@lstm_12/while/MatMul"], transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients_9/lstm_12/while/MatMul_grad/MatMul_1/StackPop, gradients_9/lstm_12/while/add_grad/Reshape_1)]]

Caused by op 'gradients_9/lstm_12/while/MatMul_grad/MatMul_1', defined at:
  File "/root/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477,

In [10]:
print ('USDT_BTC:\n', USDT_BTC)
print ('USDT_LTC:\n', USDT_LTC)
print ('USDT_ETH:\n', USDT_ETH)
print ('USDT_ETC:\n', USDT_ETC)
print ('USDT_XRP:\n', USDT_XRP)

USDT_BTC:
             predictionPrice
2017-07-21      2250.067871
USDT_LTC:
             predictionPrice
2017-07-21        42.998005
USDT_ETH:
             predictionPrice
2017-07-21       198.491074
USDT_ETC:
             predictionPrice
2017-07-21        16.313128


NameError: name 'USDT_XRP' is not defined